In [2]:
# load packages
import os
import cv2
from keras.models import model_from_json
from PIL import Image
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt

# make sure matplotlib shows images inline
%matplotlib inline

In [3]:
# set directory
os.chdir("D:/Documents/GitHub/HandSign_Recognition")

In [4]:
# define the location of the cascade files we will use to detect hands
fist_cascade_path = '00 Data/Haarcascades/fist.xml' 
palm_cascade_path =  '00 Data/Haarcascades/palm.xml'
closed_frontal_palm_cascade_path =  '00 Data/Haarcascades/closed_frontal_palm.xml'

In [5]:
# load cascade file
fistCascade = cv2.CascadeClassifier(fist_cascade_path)
palmCascade = cv2.CascadeClassifier(palm_cascade_path)
closedFrontalPalmCascade = cv2.CascadeClassifier(closed_frontal_palm_cascade_path)

In [6]:
# load json and create model
json_file = open('00 Data/my_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("00 Data/my_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [17]:
label_dict = pickle.load(open('00 Data/LabelDictionary.pkl', 'rb'))

In [35]:
# define the function to predict which letter is shown with handsigns
def predict(model, img, target_size):
    #    model: keras model
    #    img: PIL format image
    #    target_size: (width, height) tuple
    
    if img.size != target_size:
        print("the original size of the image is: " + str(img.size))
        img = img.resize(target_size)
        print("the new size of the image is: " + str(img.size))

    # convert to numpy array and preprocess
    x = np.array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x.astype(float))
    
    # make a prediction
    pred = model.predict(x)
    #print(pred)
    
    # label the prediction
    pred_label = np.argmax(pred, axis = 1)
    #print(pred_label)
    
    # translate the label to the letter, using the label dictionary
    label = list(label_dict.keys())[list(label_dict.values()).index(pred_label)]
    
    # return the label of the prediction
    return label

In [8]:
# turn on the camera
camera = cv2.VideoCapture(0)

# check if the camera is on
print("camera on: " + str(camera.isOpened()))

camera on: True


In [9]:
# as long as the camera is on, read the images
while True:
    return_value, image = camera.read()
    # check if the camera gives an image
    if return_value:
        #gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        cv2.imshow('image', image)

        # detect hands
        fists = fistCascade.detectMultiScale(image, scaleFactor = 1.1, minNeighbors = 5
                                             , minSize = (85, 64), flags = cv2.CASCADE_SCALE_IMAGE)
        palms = palmCascade.detectMultiScale(image, scaleFactor = 1.1, minNeighbors = 5
                                             , minSize = (85, 64), flags = cv2.CASCADE_SCALE_IMAGE)
        front_palms = closedFrontalPalmCascade.detectMultiScale(image, scaleFactor = 1.1, minNeighbors = 5
                                             , minSize = (85, 64), flags = cv2.CASCADE_SCALE_IMAGE)

        # check if there are hands in the image
        if len(fists) > 0 or len(palms) > 0 or len(front_palms) > 0:
            
            #print("fists: " + str(len(fists)) + ", palms: " + str(len(palms)) + ", front_palms: " + str(len(front_palms)))
            
            # convert opencv image to PIL
            pil_im = Image.fromarray(image)
            #pil_im.show()
            
            # try to define the letter from the handsign
            new_letter = predict(loaded_model, pil_im, (64,85))
            print(new_letter)

    if cv2.waitKey(1)& 0xFF == ord('s'):
        print ('stop!')
        break

# turn off the camera
camera.release()
cv2.destroyAllWindows()

fists: 0, palms: 1, front_palms: 0
fists: 0, palms: 1, front_palms: 0


KeyboardInterrupt: 

In [14]:
# turn off the camera
camera.release()
cv2.destroyAllWindows()